In [1]:
# Build Tkinter APP With the ability to be Complete Image Processing Control Panel with ability to 
from tkinter import *
from tkinter import filedialog
from PIL import ImageTk , Image
import cv2
import numpy as np

def select_image():
    global panelA, panelB, img
    
    path = filedialog.askopenfilename()
    img = cv2.imread(path)
    img = cv2.resize(img, (500,500), interpolation= cv2.INTER_LINEAR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    edged = gray
    
    image = Image.fromarray(image)  # convert the images to PIL format...
    edged = Image.fromarray(edged)
    
    image = ImageTk.PhotoImage(image)  # ...and then to tkinter format
    edged = ImageTk.PhotoImage(edged)
    
    # if the panels are None, initialize them
    if panelA is None or panelB is None:
        panelA = Label(image=image)
        panelA.image = image
        panelA.pack(side='left', padx=10, pady=10)
        panelB = Label(image=edged)
        panelB.image = edged
        panelB.pack(side='right', padx=10, pady=10)
    else:
        panelA.configure(image=image)
        panelA.image = image
        panelB.configure(image=edged)
        panelB.image = edged
        
def apply_trans():
    global img
    list_color = color.get().split()
    lower = np.array(list(map(int, list_color[:3])))
    upper = np.array(list(map(int, list_color[3:])))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    
    if sum(lower) != 0:
        edged = cv2.bitwise_and(img, img, mask=mask)
        edged = cv2.cvtColor(edged, cv2.COLOR_BGR2RGB)
    
    thresh_val = thresh.get()
    blur_val = blurr.get()
    morph_val = morph.get()
    edge_val = edge.get()
    dir_val = dirr.get()
    lower_ = lower_slide.get()
    upper_ = upper_slide.get()
    if sum(lower) == 0:
        edged = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kernel = np.ones((5,5),np.uint8)
        
        if thresh_val == 1:
            _, edged = cv2.threshold(edged,127,255,cv2.THRESH_BINARY)
        elif thresh_val == 2:
            edged = cv2.adaptiveThreshold(edged,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
        elif thresh_val ==3:
            _, edged = cv2.threshold(edged,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            
        if blur_val == 1:
            edged = cv2.blur(edged,(5,5))
        elif blur_val == 2:
            edged = blur = cv2.GaussianBlur(edged,(5,5),0)
        elif blur_val == 3:
            edged = cv2.medianBlur(edged,5)
            
        if morph_val == 1:
            edged = cv2.erode(edged, kernel, iterations = 1)
        elif morph_val == 2:
            edged = cv2.dilate(edged, kernel, iterations = 1)
        elif morph_val == 3:
            edged = cv2.morphologyEx(edged, cv2.MORPH_OPEN, kernel)
        elif morph_val == 4:
            edged = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
            
        if dir_val == 1:
            if edge_val == 1:
                edged = cv2.Sobel(edged,-1,1,0,ksize=5)
            elif edge_val == 2:
                edged = cv2.Scharr(edged,cv2.CV_64F,1,0,5)
            elif edge_val == 3:
                edged = cv2.Laplacian(edged,cv2.CV_64F,ksize=3)
            elif edge_val == 4:
                edged = cv2.Canny(edged, lower_, upper_)
        else:
            if edge_val == 1:
                edged = cv2.Sobel(edged,-1,0,1,ksize=5)
            elif edge_val == 2:
                edged = cv2.Scharr(edged,cv2.CV_64F,0,1,5)
            elif edge_val == 3:
                edged = cv2.Laplacian(edged,cv2.CV_64F,ksize=3)
            elif edge_val == 4:
                edged = cv2.Canny(edged, lower_, upper_)
            
    
    edged = Image.fromarray(edged)
    edged = ImageTk.PhotoImage(edged)
    panelB.configure(image=edged)
    panelB.image = edged
    
# initialize the window toolkit along with the two image panels
root = Tk()
#root.geometry("1750x1750")
panelA = None
panelB = None
# Button to Upload image
btn1 = Button(root, text = 'Upload Photo', command=select_image)
btn1.pack(side='top', fill='both', expand='yes', padx=10, pady=10)

left = LabelFrame(root)
left.pack(side='left', padx=10, pady=10)
# Radio Buttons for Red , Green , Blue Color Tracking or None 
label_color = LabelFrame(left, text='Color tracking', width=250)
label_color.pack(side='top', padx=10, pady=10)
color = StringVar(value='0 0 0 255 255 255')
color1 = Radiobutton(label_color, text='Red', variable=color, value='0 50 50 20 255 255')
color1.pack(anchor = W)
color2 = Radiobutton(label_color, text='Green', variable=color, value='30 50 50 90 255 255')
color2.pack(anchor = W)
color3 = Radiobutton(label_color, text='Blue', variable=color, value='100 50 50 180 255 255')
color3.pack(anchor = W)
color4 = Radiobutton(label_color, text='None', variable=color, value='0 0 0 255 255 255')
color4.pack(anchor = W)

# Radio Buttons for Binary ,Adaptive , Otsu  Threhsolding or None
label_thresh = LabelFrame(left, text='Thresholding    ', width=250)
label_thresh.pack(side='bottom', padx=10, pady=10)
thresh = IntVar(value=0)
thresh1 = Radiobutton(label_thresh, text='Binary', variable=thresh, value=1)
thresh1.pack(anchor = W)
thresh2 = Radiobutton(label_thresh, text='Adaptive', variable=thresh, value=2)
thresh2.pack(anchor = W)
thresh3 = Radiobutton(label_thresh, text='OTSU', variable=thresh, value=3)
thresh3.pack(anchor = W)
thresh4 = Radiobutton(label_thresh, text='None', variable=thresh, value=0)
thresh4.pack(anchor = W)

# Radio Buttons for Averaging , Gaussain , Median Blurring or None 
label_blur = LabelFrame(left, text='Image Blurring', width=250)
label_blur.pack(side='bottom', padx=10, pady=10)
blurr = IntVar(value=0)
blur1 = Radiobutton(label_blur, text='Averaging', variable=blurr, value=1)
blur1.pack(anchor = W)
blur2 = Radiobutton(label_blur, text='Gaussain', variable=blurr, value=2)
blur2.pack(anchor = W)
blur3 = Radiobutton(label_blur, text='Median', variable=blurr, value=3)
blur3.pack(anchor = W)
blur4 = Radiobutton(label_blur, text='None', variable=blurr, value=0)
blur4.pack(anchor = W)

# Radio Buttons for Erosion , Dilation , Openning , Closing or None
label_morph = LabelFrame(left, text='Morphology    ', width=250)
label_morph.pack(side='bottom', padx=10, pady=10)
morph = IntVar(value=0)
morph1 = Radiobutton(label_morph, text='Erosion', variable=morph, value=1)
morph1.pack(anchor = W)
morph2 = Radiobutton(label_morph, text='Dilation', variable=morph, value=2)
morph2.pack(anchor = W)
morph3 = Radiobutton(label_morph, text='Openning', variable=morph, value=3)
morph3.pack(anchor = W)
morph4 = Radiobutton(label_morph, text='Closing', variable=morph, value=4)
morph4.pack(anchor = W)
morph5 = Radiobutton(label_morph, text='None', variable=morph, value=0)
morph5.pack(anchor = W)

right = LabelFrame(root)
right.pack(side='right', padx=10, pady=10)
# Radio Buttons for Sobel , Scharr , Laplacian ,Canny or None
label_edge = LabelFrame(right, text='Edge Detection', width=250)
label_edge.pack(side='top', padx=10, pady=10)
edge = IntVar(value=0)
edge1 = Radiobutton(label_edge, text='Sobel', variable=edge, value=1)
edge1.pack(anchor = W)
edge2 = Radiobutton(label_edge, text='Scharr', variable=edge, value=2)
edge2.pack(anchor = W)
edge3 = Radiobutton(label_edge, text='Laplacian', variable=edge, value=3)
edge3.pack(anchor = W)
edge4 = Radiobutton(label_edge, text='Canny', variable=edge, value=4)
edge4.pack(anchor = W)
edge5 = Radiobutton(label_edge, text='None', variable=edge, value=0)
edge5.pack(anchor = W)

# Radio Buttons for horizontal or vertical edge detection
label_dirr = LabelFrame(right, text='Edge Direction', width=250)
label_dirr.pack(side='top', padx=10, pady=10)
dirr = IntVar(value=0)
dirr1 = Radiobutton(label_dirr, text='Horizontal', variable=dirr, value=1)
dirr1.pack(anchor = W)
dirr2 = Radiobutton(label_dirr, text='Vertical', variable=dirr, value=0)
dirr2.pack(anchor = W)

# Edge detection Canny scales
upper_slide = Scale(right, from_=70, to=200, orient=HORIZONTAL)
upper_slide.set(100)
upper_slide.pack(side="bottom", padx=10, pady=10)
lower_slide = Scale(right, from_=20, to=70, orient=HORIZONTAL)
lower_slide.set(25)
lower_slide.pack(side="bottom", padx=10, pady=10)

# Button for Applying Transformation 
btn2 = Button(root, text = 'Applying Transformation', command=apply_trans)
btn2.pack(side='bottom', fill='both', expand='yes', padx=10, pady=10)

# kick off the GUI
root.mainloop()